In [ ]:
import spacy
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')
ruler = nlp.add_pipe('entity_ruler')

In [ ]:
event_phrases = ["meet", "meeting", "scheduled", "appointment", "event", "conference", "seminar", "workshop", "presentation",
                 "gathering", "symposium", "convention", "ceremony", "forum", "session", "lecture", "webinar", "training",
                 "summit", "forum", "reception", "gala", "party", "launch", "showcase", "exhibition", "trade show",
                 "product launch", "networking", "roundtable", "panel discussion", "town hall", "townhall", "forum",
                 "discussion", "briefing", "retreat", "retreat event", "workshop", "seminar", "training session",
                 "team-building", "hackathon", "hackfest", "hack day", "competition", "contest", "exposition",
                 "business meeting", "board meeting", "annual general meeting", "stakeholder meeting", "town meeting",
                 "townhall meeting", "quarterly review", "strategy session", "brainstorming", "planning session",
                 "focus group", "launch party", "product demo", "trade fair", "expo", "fair", "festival",
                 "ceremonial event", "award ceremony", "graduation ceremony", "fundraiser", "charity event",
                 "charity gala", "fundraising event", "benefit concert", "charity auction", "volunteer event",
                 "community event", "cultural event", "art exhibition", "music festival", "film festival",
                 "food festival", "sports event", "athletic competition", "marathon", "race", "tournament"]

pattern = {"pattern": [{"LOWER": {"FUZZY2": ''}}], "label": "event"}

for event in event_phrases:
  pattern['pattern'][0]['LOWER']['FUZZY2'] = event
  ruler.add_patterns([pattern])


In [ ]:
def extract_event_details(text):
    # Process the text using spaCy
    doc = nlp(text)
    # Initialize empty dictionary to store event details
    event_details = {'Date': None, 'Time': None, 'Location': None, 'Event': False}

    # Iterate over entities and check for dates, times, and locations
    event_phrases = ["meet", "meeting", "scheduled", "appointment", "event", "conference", "seminar", "workshop", "presentation", "gathering", "symposium",
                     "convention", "ceremony", "forum", "session", "lecture", "webinar", "training", "summit", "forum", "reception", "gala", "party",
                     "launch", "showcase", "exhibition", "trade show", ""]

    spacy.displacy.render(doc, style="ent", jupyter=True)

    for ent in doc.ents:
        print(ent.label_)
        if ent.label_ == 'DATE':
            event_details['Date'] = ent.text
        elif ent.label_ == 'TIME':
            event_details['Time'] = ent.text
        elif ent.label_ == 'GPE' or ent.label_ == 'LOC':
            event_details['Location'] = ent.text
        elif ent.label_ == 'event':
            event_details['Event'] = True

    return event_details

# Example text
text = "session at 10:00 AM tomorrow with mayur"
print(extract_event_details(text))


# https://www.youtube.com/watch?v=TVBJMwSyJL8

event
TIME
{'Date': None, 'Time': '10:00 AM tomorrow', 'Location': None, 'Event': True}


In [ ]:
import spacy
from spacy.matcher import Matcher
from fuzzywuzzy import fuzz

# Load the English NLP model from spaCy
nlp = spacy.load("en_core_web_sm")

def is_event(token):
    # Function to check if a token is an event using high-level fuzzy matching
    event_keywords = ["meet", "meeting", "scheduled", "appointment", "event"]  # Add more keywords as needed
    for keyword in event_keywords:
        if fuzz.ratio(token.lower_, keyword.lower()) > 90:  # Adjust the threshold as needed (e.g., 90)
            return True
    return False

def extract_event_details(text):
    # Process the text using spaCy
    doc = nlp(text)

    # Initialize empty dictionary to store event details
    event_details = {'Date': None, 'Time': None, 'Location': None, 'Event': False}

    # Define a pattern for event phrases using spaCy's Matcher
    event_pattern = [{"TEXT": {"FUZZY": {"IN": ["meet", "meeting", "scheduled", "appointment", "event"]}}}]
    matcher = Matcher(nlp.vocab)
    matcher.add("EVENT_PATTERN", [event_pattern])

    # Use the matcher to find matches in the text
    matches = matcher(doc)

    # If there is a match, set 'Event' to True
    if matches:
        event_details['Event'] = True

    # Iterate over entities and check for dates, times, and locations
    for ent in doc.ents:
        if ent.label_ == 'DATE':
            event_details['Date'] = ent.text
        elif ent.label_ == 'TIME':
            event_details['Time'] = ent.text
        elif ent.label_ in ['GPE', 'LOC']:
            event_details['Location'] = ent.text

    return event_details

# Example text
text = "For identifying the issue with the tree health, we organize an event on this day."
event_info = extract_event_details(text)
print(event_info)


{'Date': 'this day', 'Time': None, 'Location': None, 'Event': True}
